In [3]:
import os
# from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_ollama import ChatOllama
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, Tool

os.environ["TAVILY_API_KEY"] = "tavily key"
os.environ["SERPAPI_API_KEY"] = "serpapi key"
os.environ["GOOGLE_API_KEY"] = "google api key"
os.environ["LANGSMITH_API"] = "langsmith key"
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN-API-KEY"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_PROJECT"] = "Explore Evaluating index using LLM" 

from langchain.agents import Tool 


llm = ChatOllama(
    model = "llama3.1",
    base_url = "http://localhost:11434")




def get_function_tools():
  search = TavilySearchAPIWrapper()
  tavily_tool = TavilySearchResults(api_wrapper=search)

  tools = [
      tavily_tool
  ]

  tools.extend(load_tools(['wikipedia']))

  return tools

In [6]:
prompt = hub.pull("hwchase17/react")
tools = get_function_tools()
agent = create_react_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

output = agent_executor.invoke({"input": "Who is nore popular, Da Vinci or Picasso?"})
process_chunks(output)



> Entering new AgentExecutor chain...
Thought: To compare the popularity of Leonardo da Vinci and Pablo Picasso, I need to gather information about their public recognition and influence.
Action: tavily_search_results_json
Action Input: "Leonardo da Vinci vs Pablo Picasso in terms of popularity[{'url': 'https://anitalouiseart.com/differences-between-leonardo-da-vinci-and-pablo-picasso/', 'content': 'Leonardo da Vinci and Pablo Picasso lived in entirely different times; their artwork is also very different. Both are considered artistic geniuses who influenced art movements and were essential artists during the time and place they lived. Leonardo only painted a few paintings, whereas Picasso was a prolific artist.'}, {'url': 'https://www.miicreative.com/blogs/painting/art-history-the-main-differences-between-picasso-and-da-vinci', 'content': 'Picasso was more lyrical, whereas Leonardo was more philosophical. The Renaissance emphasized elegance, but the early twentieth century needed in

KeyboardInterrupt: 

In [8]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from rich.console import Console

# Initialize dotenv to load environment variables
load_dotenv()

# Initialize Rich for better output formatting and visualization
rich = Console()

# Initialize OpenAI LLM

# Initialize Tavily
tavily = TavilySearchResults(max_results=3)

# Create a LangGraph agent
langgraph_agent = create_react_agent(model=llm, tools=[tavily])


# Define a function to process chunks from the agent
def process_chunks(chunk):
    """
    Process a chunk of data and extract information about tool calls made by the agent.

    Parameters:
        chunk (dict): A dictionary containing information about the agent's messages.

    Returns:
        None

    This function processes a chunk of data and checks if it contains information about an agent. If the chunk contains
    an agent's message, it iterates over the messages in the agent's messages. For each message, it checks if the
    message contains tool calls. If a tool call is found, the function extracts the tool name and query from the
    message and prints a formatted message using the Rich library. If no tool call is found, the function extracts
    the agent's answer from the message and prints it using the Rich library.

    The function uses the Rich library for formatting and printing the messages.
    """

    # Check if the chunk contains an agent's message
    if "agent" in chunk:
        # Iterate over the messages in the chunk
        for message in chunk["agent"]["messages"]:
            # Check if the message contains tool calls
            if "tool_calls" in message.additional_kwargs:
                # If the message contains tool calls, extract and display an informative message with tool call details

                # Extract all the tool calls
                tool_calls = message.additional_kwargs["tool_calls"]

                # Iterate over the tool calls
                for tool_call in tool_calls:
                    # Extract the tool name
                    tool_name = tool_call["function"]["name"]

                    # Extract the tool query
                    tool_arguments = eval(tool_call["function"]["arguments"])
                    tool_query = tool_arguments["query"]

                    # Display an informative message with tool call details
                    rich.print(
                        f"\nThe agent is calling the tool [on deep_sky_blue1]{tool_name}[/on deep_sky_blue1] with the query [on deep_sky_blue1]{tool_query}[/on deep_sky_blue1]. Please wait for the agent's answer[deep_sky_blue1]...[/deep_sky_blue1]",
                        style="deep_sky_blue1",
                    )
            else:
                # If the message doesn't contain tool calls, extract and display the agent's answer

                # Extract the agent's answer
                agent_anser = message.content

                # Display the agent's answer
                rich.print(f"\nAgent:\n{agent_anser}", style="black on white")


# Loop until the user chooses to quit the chat
while True:
    # Get the user's question and display it in the terminal
    user_question = input("\nUser:\n")

    # Check if the user wants to quit the chat
    if user_question.lower() == "quit":
        rich.print("\nAgent:\nHave a nice day! :wave:\n", style="black on white")
        break

    # Use the stream method of the LangGraph agent to get the agent's answer
    for chunk in langgraph_agent.stream(
        {"messages": [HumanMessage(content=user_question)]}
    ):
        # Process the chunks from the agent
        process_chunks(chunk)



User:
 Who is nore popular, Da Vinci or Picasso?


Agent:

Agent:
Based on the tool call results, it appears that both Leonardo da Vinci and Pablo Picasso are highly influential 
artists in their respective times and styles. However, if I had to determine who is more popular based on the 
provided information, I would say that Pablo Picasso may have a slight edge.

The search results suggest that Picasso was a more prolific artist than Da Vinci, with a greater number of works 
produced throughout his career. Additionally, the articles mention that Picasso's style and impact were more suited
to the era of Modernism, which may have contributed to his broader appeal and recognition.

That being said, it's essential to note that both artists are widely regarded as geniuses in their respective 
fields, and their contributions to art history are invaluable. Ultimately, the question of who is "more popular" is
subjective and may depend on individual perspectives and criteria for evaluation.


User:
 bye


Agent:


KeyboardInterrupt

